# Preparation

## Mount Google Drive

In [ ]:
!pip3 install google

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

## Go to working space

In [ ]:
import os
os.chdir('drive/colab/course')

## Download test images

In [ ]:
!wget https://raw.githubusercontent.com/BVLC/caffe/master/examples/images/cat_gray.jpg 

In [ ]:
!wget https://raw.githubusercontent.com/BVLC/caffe/master/examples/images/cat.jpg

## Install image library

In [ ]:
!pip3 install Pillow

# Convolution over gray-scale images

## Naive implementation

In [ ]:
import numpy as np

In [ ]:
def conv_gray1(x, W, p=0, s=1):
  h, w = x.shape
  k = W.shape[0]
  oh = (h + p -k) // s + 1
  ow = (w + p -k) // s + 1
  
  out = np.zeros((oh, ow))
  for i in range(oh):
    for j in range(ow):
      for a in range(k):
        for b in range(k):
          out[i, j] += x[i+a, j+b] * W[a, b]
  return out  

In [ ]:
import PIL
img=np.asarray(PIL.Image.open('cat_gray.jpg'))
print(img.shape)

Test the running time 

In [ ]:
import time
W = np.array([[0.5, -1], [1, -1]])
tick = time.time()
out = conv_gray1(img, W)
print('time for conv_gray1: %f' % (time.time() - tick))

In [ ]:
print(out.max(), out.min())

Display the output feature map

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(np.uint8(out/out.max()*255), cmap='gray')
plt.grid(False)

## Advanced implementation

In [ ]:
def conv_gray2(x, W, p=0, s=1):
  h, w = x.shape
  k = W.shape[0]
  oh = (h + p -k) // s + 1
  ow = (w + p -k) // s + 1
  
  fields = np.zeros((oh, ow, k, k))  
  
  
  for i in range(oh):
    for j in range(ow):
      fields[i, j]= x[i*s:i*s+k, j*s:j*s+k]
  out = np.dot(fields.reshape(oh*ow, -1), W.flatten())
  
  return out.reshape((oh, ow))

Test the convolution time

In [ ]:
tick = time.time()
out = conv_gray2(img, W)
print('time for conv_gray2 %f' % (time.time() - tick))

Display the output feature maps

In [ ]:
plt.imshow(np.uint8(out/out.max()*255), cmap='gray')
plt.grid(False)

# Convolution over RGB images

##  Forward

In [ ]:
 def conv_rgb(x, W, p=0, s=1):
    c, h, w = x.shape
    n, _, k, _ = W.shape
    oh = (h + p -k) // s + 1
    ow = (w + p -k) // s + 1
  
    fields = np.zeros((oh, ow, c*k*k))  
  
    x_pad = np.zeros((c, h+p, w+p))
    x_pad[:, (p//2):h+(p//2), (p//2):w+(p//2)] = x
  
    for i in range(oh):
      for j in range(ow):
        fields[i, j]= x_pad[:, i*s:i*s+k, j*s:j*s+k].flatten()
    W = W.reshape((n, c*k*k))
    fields = fields.reshape(oh*ow, c*k*k).T
    out = np.dot(W, fields)
  
    return out.reshape((n, oh, ow))

In [ ]:
img=np.asarray(PIL.Image.open('cat.jpg'))
print(img.shape)

In [ ]:
img=img.transpose(2, 0, 1)
print(img.shape)

In [ ]:
W=np.random.randn(3, 3, 3, 3) / 10
out = conv_rgb(img, W)


In [ ]:
out_img = out.transpose(1, 2, 0)


In [ ]:
print(out_img.shape)

In [ ]:
plt.imshow(np.uint8(out_img/out_img.max()*255))
plt.grid(False)

## Backward

In [ ]:
def conv_rgb_back(x, W, dy, p=0, s=1):
    c, h, w = x.shape    
    n, oh, ow = dy.shape
    k = W.shape[2]
      
    fields = np.zeros((oh, ow, c*k*k)) 
    
    x_pad = np.zeros((c, h+p, w+p))
    x_pad[:, (p//2):h+(p//2), (p//2):w+(p//2)] = x
  
  
    for i in range(oh):
      for j in range(ow):
        fields[i, j, :]= x_pad[:, i*s:i*s+k, j*s:j*s+k].flatten()
    fields = fields.reshape(oh*ow, -1).T
    W = W.reshape((n, c*k*k))
    dy = dy.reshape((n, oh*ow))
    dx_ = np.dot(W.T, dy).reshape((c*k*k, oh, ow))
    dW = np.dot(dy, fields.T)
    
    dx = np.zeros(x_pad.shape)
    for i in range(oh):
      for j in range(ow):
        dx[:, i*s:i*s+k, j*s:j*s+k] += dx_[:, i, j].reshape(c, k, k)
  
    return dx[:, (p//2):h+(p//2), (p//2):w+(p//2), dW.reshape((n, c, k, k))

In [ ]:
dy = np.random.rand(out.shape[0], out.shape[1], out.shape[2])
dx, dW = conv_rgb_back(img, W, dy)

# MaxPooling

In [ ]:
class MaxPooling(object):
  
  def forward(self, x, k, p, s):
    c, h, w = x.shape    
    oh = (h + p -k) // s + 1
    ow = (w + p -k) // s + 1
  
    fields = np.zeros((oh, ow, c, k * k))  
    x_pad = np.zeros((c, h+p, w+p))
    x_pad[:, (p//2):h+(p//2), (p//2):w+(p//2)] = x
  
    for i in range(oh):
      for j in range(ow):
        fields[i, j]= x[:, i*s:i*s+k, j*s:j*s+k].reshape((c, -1))
    
    out = np.max(fields, axis=3)
    
    return out.transpose(2, 0, 1)
    
    
  def backward(self, dy):